In [1]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [23]:
df = pd.read_csv("OL-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
#1092

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted,hasstats,awards
0,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85,N,0.0,0.0
1,1997,brad badger,Stanford,OG,76.4,296,5.22,21.0,32.5,111.0,4.13,NaN,Y,0.0,0.0
2,1997,ben bordelon,Louisiana State,OT,77.1,298,5.42,22.0,28.0,104.0,4.73,8.25,N,0.0,0.0
3,1997,lamont burns,East Carolina (NC),OG,75.6,299,5.32,30.0,28.5,109.0,4.70,NaN,Y,0.0,0.0
4,1997,calvin collins,Texas A&M,OG,73.6,307,5.19,22.0,28.0,102.0,4.61,NaN,Y,0.0,0.0


In [11]:
df["hasstats"] = ""
df["awards"] = ""

In [5]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [24]:
for index, row in df.iterrows():
    if (df.loc[index,"hasstats"] != 1.0):
        if (df.loc[index,"hasstats"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if (len(browser.find_by_css("h1[itemprop='name']")) > 0):
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"awards"] = 1
                else:
                    df.loc[index,"awards"] = 0
                df.loc[index,"hasstats"] = 1
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"awards"] = 0
                df.loc[index,"hasstats"] = 0
            else:
                listed = browser.find_by_css("div.search-item")            
                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) > 0):
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"awards"] = 1
                            else:
                                df.loc[index,"awards"] = 0
                            df.loc[index,"hasstats"] = 1
                            break
                        else:
                            df.loc[index,"awards"] = 0
                            df.loc[index,"hasstats"] = 0
                            break
                    else:
                        df.loc[index,"awards"] = 0
                        df.loc[index,"hasstats"] = 0

In [25]:
df.to_csv("OL-Train.csv", index=False)